# 03. Ch 03. Data Preparation - 03. 실습 데이터 살펴보기

In [ ]:
셔플링 후 트레인 밸리드 테스트 셋으로 나눠주기
이후 토크나이제이션

# 04. Ch 03. Data Preparation - 04. Review Preprocessing

In [ ]:
<전처리 워크플로>
데이터 수집 -> 정제 -> 레이블링 -> 토크나이제이션 -> 서브워드 세그멘테이션 -> Batchify

토크나이제이션만 하거나 서브워드 세그멘테이션만 하거나 둘 다 하거나
프리트레인 랭귀지 모델 같은 경우에는 서브워드 세그멘테이션만 하는 경우도 많다.
둘 다 하는 걸 추천
애초 영어 같은 경우에는 띄어쓰기가 잘 돼 있다.
한국어 교착어고 띄어쓰기가 개판이라는 언어적 특징이 있기 떄문에 둘 다 하는 것을 선호한다는 것.
성능의 미미한 차이가 있을 뿐. 했을 때 안 되던게 되거나 안 했을 때 되던 게 안 되는 일은 벌어지지 않음.

# 05. Ch 03. Data Preparation - 05. 실습 Tokenization

In [ ]:
mecab(토크나이제이션) 후 BPE(서브워드 세그멘테이션) 할 것.
데이터는 이미 정제까지 돼 있는 상태.

ll = ls -al 
head -n 5 ./data/corpus.shuf.train.ko | mecab -0 wakati
cat ./data/corpus.shuf.test.ko | mecab -0 wakati > ./data/corpus.shuf.test.tok.ko

cat ./data/corpus.shuf.test.ko | mecab -0 wakati -b 99999 | python ./post_tokenize.py ./data/corpus.shuf.test.ko
> ./data/corpus.shuf.test.tok.ko
#b: 버퍼 사이즈

head -n 5 ./data/corpus.shuf.test.tok.ko | python detokenizer.py

aihub에서 엑셀 파일 받고 -> 텍스트 파일로 옮기고 -> 탭 명령어를 통해 corpus.tsv 하나로 합치고 -> 셔플링
-> 120만/20만/20만 테스트 밸리드 트레인셋 나누고 -> cut을 통해 ko en 이라는 확장자 파일 만든 다음 -> 토크나이제이션 수행

명령어 뒤에 & 붙이면 병렬로 실행할 수 있다.
jobs 명령어 치면 현재 돌아가고 있는 작업 볼 수 있음.

wc -l ./data/corpus.shuf.*tok.en

# 06. Ch 03. Data Preparation - 06. 실습 Subword Segmentaion

In [ ]:
정제가 잘 된 코퍼스는 BPE만 먹여도 잘 될 것이다
그런데 일반적으로 크롤링을 해서 얻은 코퍼스는 띄어쓰기가 굉장히 잘 안 돼 있음
이런 코퍼스에 바로 BPE를 먹이면 띄어쓰기에 대해서 통계적으로 BPE가 들어가게 되는건데
노이즈가 껴서 통계가 고르지 않게될 것이다.
뜻하지 않은 대로 세그멘테이션 할 수 있기 때문에 토크나이제이션(mecab) 하고 나서 다음에 서브워드 세그멘테이션 하는 것을 추천.
영어는 쉼표나 마침표는 같은 것만 띄어주고… 영어는 워낙 띄어쓰기 잘 돼 있어서.

learn_bpe.py와 apply_bpe.py를 주로 사용할 것

트레인 파일에 대해 학습해 트레인 셋에 그대로 적용하고
밸리드와 테스트 셋에도 트레인셋에서 배운 것을 그대로 적용한다.
이게 정석
전체 파일에 대해 학습해 통계를 내면 테스트 셋에 대한 통계까지 모두 공개가 되는 것.

history | grep apply_bpe